<a href="https://colab.research.google.com/github/vijayk2000/capstone_project/blob/main/Pinacle_capstone_embeddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# %pip install langchain_chroma
# %pip install langchain_openai
# %pip install langchain_google_genai
# %pip install langchain_huggingface
# %pip install langchain_community
# %pip install FlagEmbedding
# !pip install python-dotenv
# %pip install pypdf

In [ ]:
from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain.docstore.document import Document

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Load the files and index them in a vector database


In [ ]:
docs= []
for doc in PyPDFDirectoryLoader("/content/data").load():


    metadata ={
        "source": doc.metadata["source"],
        "page": doc.metadata["page"],

    }
    doc_text = ' '.join(doc.page_content.split())
    # Remove newlines and extra spaces
    docs.append(Document(page_content=doc_text,
                                   metadata=metadata))

# PyPDFDirectoryLoader loads all PDFs in a directory

In [ ]:
from langchain_openai import OpenAIEmbeddings
from langchain_google_genai import GoogleGenerativeAIEmbeddings
#from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from dotenv import load_dotenv
import os
from langchain_chroma import Chroma
from FlagEmbedding import BGEM3FlagModel
# The splitting and chunking strategy
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings

load_dotenv(".env")
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
os.environ["GOOGLE_API_KEY"] = os.getenv("GOOGLE_API_KEY")
os.environ['HUGGINGFACEHUB_API_TOKEN'] = os.getenv('HUGGINGFACE_TOKEN')

In [ ]:
splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)

In [ ]:
# Download the embeddings
open_ai_large = OpenAIEmbeddings(model="text-embedding-3-large")
open_ai_small = OpenAIEmbeddings(model="text-embedding-3-small")


model_name = "sentence-transformers/all-mpnet-base-v2"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': False}
hf = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.4k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
def create_chroma_db(docs, embedding_list, embedding_names, similarity_type="cosine"):
  splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
  chunked_docs = splitter.split_documents(docs)

  for embedding, name in zip(embedding_list, embedding_names):
    for similarity in similarity_type:
      persist_dir = f"./capstone_db_{name}_{similarity}"
      chromba_db = Chroma.from_documents(documents=chunked_docs,
                                        collection_name=f'capstone_db_{name}_{similarity}',
                                        embedding=embedding,
                                        collection_metadata={"hnsw:space": similarity},
                                         persist_directory=persist_dir)

      print(f"Chroma DB created for embedding: {name} with similarity: {similarity} in directory: {persist_dir}")

In [ ]:
docs = docs
embedding_list = [open_ai_large, open_ai_small, hf]
embedding_names = ["open_ai_large", "open_ai_small","mp_allnet"]
similarity_type = ["cosine", "l2"]

In [ ]:
create_chroma_db(docs=docs, embedding_list=embedding_list, embedding_names=embedding_names, similarity_type=similarity_type)

Chroma DB created for embedding: open_ai_large with similarity: cosine in directory: ./capstone_db_open_ai_large_cosine
Chroma DB created for embedding: open_ai_large with similarity: l2 in directory: ./capstone_db_open_ai_large_l2
Chroma DB created for embedding: open_ai_small with similarity: cosine in directory: ./capstone_db_open_ai_small_cosine
Chroma DB created for embedding: open_ai_small with similarity: l2 in directory: ./capstone_db_open_ai_small_l2
Chroma DB created for embedding: mp_allnet with similarity: cosine in directory: ./capstone_db_mp_allnet_cosine
Chroma DB created for embedding: mp_allnet with similarity: l2 in directory: ./capstone_db_mp_allnet_l2


In [ ]:
# embedding_name = []
# similarity_types = []
# retriever_details = []

In [ ]:
for embedding, name in zip(embedding_list, embedding_names):

  for similarity  in similarity_type:

    chroma_db = Chroma(persist_directory=f"./capstone_db_{name}_{similarity}",
                        collection_name=f'capstone_db_{name}_{similarity}',
                         embedding_function=embedding)

    similarity_retriever = chroma_db.as_retriever(search_type="similarity_score_threshold",
                                                search_kwargs={"k": 5, "score_threshold": 0.0})
    embed_search = similarity_retriever.invoke("what is attention layer", k=5)
    embedding_name.append(name)

    retriever_details.append(embed_search)
    similarity_type.append(similarity)


print((embedding_name, similarity_type))

AttributeError: 'str' object has no attribute 'append'

In [ ]:
embedding_list = [open_ai_large, open_ai_small, hf]
embedding_names = ["open_ai_large", "open_ai_small","mp_allnet"]
similarity_type = ["cosine", "l2"]

In [ ]:
# embedding_names =[]
# retriever_details = []
# similarity_types = []
combined_list_results = []

def find_best_embedding(embedding_list, embedding_names, similarity_type, question):
  # load from disk
  results = []

  for embedding, name in zip(embedding_list, embedding_names):
    for similarity  in similarity_type:
      print(embedding)
      print(name)
      print(similarity)

      chroma_db = Chroma(persist_directory=f"./capstone_db_{name}_{similarity}",
                        collection_name=f'capstone_db_{name}_{similarity}',
                         embedding_function=embedding)

      similarity_retriever = chroma_db.as_retriever(search_type="similarity_score_threshold",
                                                search_kwargs={"k": 5, "score_threshold": 0.0})
      embed_search = similarity_retriever.invoke(question, k=5)

    combined_list_results.append({
        "embedding_name": name,
        "similarity_type": similarity,
        "retriever_details": embed_search
    })

  return combined_list_results

## Analyze Cosine  Similarity

In [ ]:
ques="what is attention layer"
output  = find_best_embedding(embedding_list = embedding_list, embedding_names = embedding_names,
                              similarity_type = similarity_type, question = ques)
print(output)

client=<openai.resources.embeddings.Embeddings object at 0x7b1d04593610> async_client=<openai.resources.embeddings.AsyncEmbeddings object at 0x7b1d03f449d0> model='text-embedding-3-large' dimensions=None deployment='text-embedding-ada-002' openai_api_version=None openai_api_base=None openai_api_type=None openai_proxy=None embedding_ctx_length=8191 openai_api_key=SecretStr('**********') openai_organization=None allowed_special=None disallowed_special=None chunk_size=1000 max_retries=2 request_timeout=None headers=None tiktoken_enabled=True tiktoken_model_name=None show_progress_bar=False model_kwargs={} skip_empty=False default_headers=None default_query=None retry_min_seconds=4 retry_max_seconds=20 http_client=None http_async_client=None check_embedding_ctx_length=True
open_ai_large
cosine
client=<openai.resources.embeddings.Embeddings object at 0x7b1d04593610> async_client=<openai.resources.embeddings.AsyncEmbeddings object at 0x7b1d03f449d0> model='text-embedding-3-large' dimensions=

## Analyzing Cosine Similarity results

In [ ]:
#Iterate through the list of dictionaries returned by find_best_embedding
for item in output:
    embedding_name = item["embedding_name"]
    similarity_search = "cosine"  #item["similarity_type"] # Renamed to avoid conflict and be more descriptive
    retriever = item["retriever_details"]

    print(f"Embedding: {embedding_name}")
    print(f"Similarity_search: {similarity_search}")
    for doc in retriever:
        print(f"Document: {doc.page_content}")
        #print(f"Score: {doc.metadata['similarity_score']}") # Uncomment if score is available
        print(f"Source: {doc.metadata['source']}")
        print(f"Page: {doc.metadata['page']}")
        print("-" * 80)

Embedding: open_ai_large
Similarity_search: cosine
Document: layer connects all positions with a constant number of sequentially executed operations, whereas a recurrent layer requires O(n) sequential operations. In terms of computational complexity, self-attention layers are faster than recurrent layers when the sequence 6
Source: /content/data/attention_paper.pdf
Page: 5
--------------------------------------------------------------------------------
Document: of Attention in our Model The Transformer uses multi-head attention in three different ways: • In "encoder-decoder attention" layers, the queries come from the previous decoder layer, and the memory keys and values come from the output of the encoder. This allows every position in the decoder to attend over all positions in the input sequence. This mimics the typical encoder-decoder attention mechanisms in sequence-to-sequence models such as [38, 2, 9]. • The encoder contains self-attention layers. In a self-attention layer all

In [ ]:
#Iterate through the list of dictionaries returned by find_best_embedding
for item in output:
    embedding_name = item["embedding_name"]
    similarity_search = "l2"  #item["similarity_type"] # Renamed to avoid conflict and be more descriptive
    retriever = item["retriever_details"]

    print(f"Embedding: {embedding_name}")
    print(f"Similarity_search: {similarity_search}")
    for doc in retriever:
        print(f"Document: {doc.page_content}")
        #print(f"Score: {doc.metadata['similarity_score']}") # Uncomment if score is available
        print(f"Source: {doc.metadata['source']}")
        print(f"Page: {doc.metadata['page']}")
        print("-" * 80)

Embedding: open_ai_large
Similarity_search: l2
Document: layer connects all positions with a constant number of sequentially executed operations, whereas a recurrent layer requires O(n) sequential operations. In terms of computational complexity, self-attention layers are faster than recurrent layers when the sequence 6
Source: /content/data/attention_paper.pdf
Page: 5
--------------------------------------------------------------------------------
Document: of Attention in our Model The Transformer uses multi-head attention in three different ways: • In "encoder-decoder attention" layers, the queries come from the previous decoder layer, and the memory keys and values come from the output of the encoder. This allows every position in the decoder to attend over all positions in the input sequence. This mimics the typical encoder-decoder attention mechanisms in sequence-to-sequence models such as [38, 2, 9]. • The encoder contains self-attention layers. In a self-attention layer all of 

## Cosine similarity is better than Euclidean similarity.

In [ ]:
ques="what is gemini"
output  = find_best_embedding(embedding_list = embedding_list, embedding_names = embedding_names,
                              similarity_type = similarity_type, question = ques)
#print(output)

client=<openai.resources.embeddings.Embeddings object at 0x7b1d04593610> async_client=<openai.resources.embeddings.AsyncEmbeddings object at 0x7b1d03f449d0> model='text-embedding-3-large' dimensions=None deployment='text-embedding-ada-002' openai_api_version=None openai_api_base=None openai_api_type=None openai_proxy=None embedding_ctx_length=8191 openai_api_key=SecretStr('**********') openai_organization=None allowed_special=None disallowed_special=None chunk_size=1000 max_retries=2 request_timeout=None headers=None tiktoken_enabled=True tiktoken_model_name=None show_progress_bar=False model_kwargs={} skip_empty=False default_headers=None default_query=None retry_min_seconds=4 retry_max_seconds=20 http_client=None http_async_client=None check_embedding_ctx_length=True
open_ai_large
cosine
client=<openai.resources.embeddings.Embeddings object at 0x7b1d04593610> async_client=<openai.resources.embeddings.AsyncEmbeddings object at 0x7b1d03f449d0> model='text-embedding-3-large' dimensions=

In [ ]:
#Iterate through the list of dictionaries returned by find_best_embedding
for item in output:
    embedding_name = item["embedding_name"]
    similarity_search = "cosine"  #item["similarity_type"] # Renamed to avoid conflict and be more descriptive
    retriever = item["retriever_details"]

    print(f"Embedding: {embedding_name}")
    print(f"Similarity_search: {similarity_search}")
    for doc in retriever:
        print(f"Document: {doc.page_content}")
        #print(f"Score: {doc.metadata['similarity_score']}") # Uncomment if score is available
        print(f"Source: {doc.metadata['source']}")
        print(f"Page: {doc.metadata['page']}")
        print("-" * 80)

Embedding: open_ai_large
Similarity_search: cosine
Document: layer connects all positions with a constant number of sequentially executed operations, whereas a recurrent layer requires O(n) sequential operations. In terms of computational complexity, self-attention layers are faster than recurrent layers when the sequence 6
Source: /content/data/attention_paper.pdf
Page: 5
--------------------------------------------------------------------------------
Document: of Attention in our Model The Transformer uses multi-head attention in three different ways: • In "encoder-decoder attention" layers, the queries come from the previous decoder layer, and the memory keys and values come from the output of the encoder. This allows every position in the decoder to attend over all positions in the input sequence. This mimics the typical encoder-decoder attention mechanisms in sequence-to-sequence models such as [38, 2, 9]. • The encoder contains self-attention layers. In a self-attention layer all

In [ ]:
ques="what are the conclusions of llms"
output  = find_best_embedding(embedding_list = embedding_list, embedding_names = embedding_names,
                              similarity_type = similarity_type, question = ques)
#print(output)

client=<openai.resources.embeddings.Embeddings object at 0x7b1d04593610> async_client=<openai.resources.embeddings.AsyncEmbeddings object at 0x7b1d03f449d0> model='text-embedding-3-large' dimensions=None deployment='text-embedding-ada-002' openai_api_version=None openai_api_base=None openai_api_type=None openai_proxy=None embedding_ctx_length=8191 openai_api_key=SecretStr('**********') openai_organization=None allowed_special=None disallowed_special=None chunk_size=1000 max_retries=2 request_timeout=None headers=None tiktoken_enabled=True tiktoken_model_name=None show_progress_bar=False model_kwargs={} skip_empty=False default_headers=None default_query=None retry_min_seconds=4 retry_max_seconds=20 http_client=None http_async_client=None check_embedding_ctx_length=True
open_ai_large
cosine
client=<openai.resources.embeddings.Embeddings object at 0x7b1d04593610> async_client=<openai.resources.embeddings.AsyncEmbeddings object at 0x7b1d03f449d0> model='text-embedding-3-large' dimensions=

In [ ]:
#Iterate through the list of dictionaries returned by find_best_embedding
for item in output:
    embedding_name = item["embedding_name"]
    similarity_search = "cosine"  #item["similarity_type"] # Renamed to avoid conflict and be more descriptive
    retriever = item["retriever_details"]

    print(f"Embedding: {embedding_name}")
    print(f"Similarity_search: {similarity_search}")
    for doc in retriever:
        print(f"Document: {doc.page_content}")
        #print(f"Score: {doc.metadata['similarity_score']}") # Uncomment if score is available
        print(f"Source: {doc.metadata['source']}")
        print(f"Page: {doc.metadata['page']}")
        print("-" * 80)

Embedding: open_ai_large
Similarity_search: cosine
Document: layer connects all positions with a constant number of sequentially executed operations, whereas a recurrent layer requires O(n) sequential operations. In terms of computational complexity, self-attention layers are faster than recurrent layers when the sequence 6
Source: /content/data/attention_paper.pdf
Page: 5
--------------------------------------------------------------------------------
Document: of Attention in our Model The Transformer uses multi-head attention in three different ways: • In "encoder-decoder attention" layers, the queries come from the previous decoder layer, and the memory keys and values come from the output of the encoder. This allows every position in the decoder to attend over all positions in the input sequence. This mimics the typical encoder-decoder attention mechanisms in sequence-to-sequence models such as [38, 2, 9]. • The encoder contains self-attention layers. In a self-attention layer all

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

prompt = """You are an assistant for question-answering tasks.
            Use the following pieces of retrieved context to answer the question.
            If the answer is not present in the context, just say that you don't know.
            Keep the answer to the point. also show the top 3 context documents of the answer.

            Question:
            {question}

            Context:
            {context}

            Answer:
         """

prompt_template = ChatPromptTemplate.from_template(prompt)

In [ ]:
from langchain_openai import ChatOpenAI

chatgpt = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)


In [ ]:
from langchain_core.runnables import RunnablePassthrough

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

qa_rag_chain = (
    {
        "context": (similarity_retriever
                      |
                    format_docs),
        "question": RunnablePassthrough()
    }
       |
    prompt_template
      |
    chatgpt
)

In [ ]:
from IPython.display import display, Markdown
query = "What is gemini"
result = qa_rag_chain.invoke(query)
display(Markdown(result.content))


Gemini is a family of highly capable multimodal models that includes evaluations at both the model and product levels, focusing on safety mitigations and user experience. It addresses critical policy areas such as hate speech and dangerous content, and adopts a user-centric approach to maximize diversity across various topics and user journeys.

Top 3 context documents:
1. Gemini: A Family of Highly Capable Multimodal Models 7.4.2. Gemini Advanced
2. Gemini: A Family of Highly Capable Multimodal Models 7.4.2. Gemini Advanced
3. Gemini: A Family of Highly Capable Multimodal Models 7.4.2. Gemini Advanced